# Plot raw data

**Date:** July 2019

**Purpose:** create a plot of the data pr cycle with voltage on one y-axis and current on the other where the individual steps are within each span with added labels showing C-rate and values from the step_table

In [ ]:
import os
import itertools

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import bokeh
import cellpy
from cellpy.readers.core import xldate_as_datetime

In [ ]:
%matplotlib inline
pd.options.display.max_columns = 100

In [ ]:
filename = "../testdata/hdf5/20160805_test001_45_cc.h5"
assert os.path.isfile(filename)
my_cell = cellpy.cell(filename)

## Checking data

In [ ]:
my_cell.make_step_table()

In [ ]:
data = my_cell.dataset.dfdata
table = my_cell.dataset.step_table
summary = my_cell.dataset.dfsummary

In [ ]:
data.columns

In [ ]:
v1 = my_cell.get_voltage(1)

In [ ]:
def get_datetime(cell, cycle=1):
    mask = cell.dataset.dfdata['Cycle_Index'] == cycle
    dt = cell.dataset.dfdata.loc[mask, "DateTime"]
    return dt


In [ ]:
d1 = get_datetime(my_cell)

In [ ]:
t1 = my_cell.get_timestamp(1)

In [ ]:
c1 = my_cell.get_current(1, )

In [ ]:
n = my_cell.get_cycle_numbers()

In [ ]:
table.head(3)

In [ ]:
summary.head(3)

In [ ]:
data.head(3)

In [ ]:
cycle = 1
step = 5

In [ ]:
m_table = (table.cycle==cycle) & (table.step==step)

In [ ]:
p1, p2 = table.loc[m_table, ['point_min', 'point_max']].values[0]

In [ ]:
def plot_step(ax, x, y, color):
    ax.plot(x, y, color=color, linewidth=3)
    

In [ ]:
def get_info(table, cycle, step):
    m_table = (table.cycle==cycle) & (table.step==step)
    p1, p2 = table.loc[m_table, ['point_min', 'point_max']].values[0]
    c1, c2 = table.loc[m_table, ['current_min', 'current_max']].abs().values[0]
    d_voltage, d_current  = table.loc[m_table, ['voltage_delta', 'current_delta']].values[0]
    d_discharge, d_charge  = table.loc[m_table, ['discharge_delta', 'charge_delta']].values[0]
    current_max = (c1 + c2)/2
    rate = 12
    step_type = table.loc[m_table, 'type'].values[0]
    return [step_type, rate, current_max, d_voltage, d_current, d_discharge, d_charge]

In [ ]:
def plot_raw_cycle(cell, cycle):
    data = cell.dataset.dfdata
    table = cell.dataset.step_table

    span_colors = [
        "#4682B4",
        "#FFA07A"  
    ]

    voltage_color = "#008B8B"
    current_color = "#CD5C5C"

    m_cycle_data = (data.Cycle_Index==cycle) 
    all_steps = data[m_cycle_data]["Step_Index"].unique()

    color=itertools.cycle(span_colors)

    fig = plt.figure(figsize=(20,8))
    fig.suptitle(f"Cycle: {cycle}")

    ax3 = plt.subplot2grid((8, 3), (0, 0), colspan=3, rowspan=1, fig=fig)  # steps
    ax4 = plt.subplot2grid((8, 3), (1, 0), colspan=3, rowspan=2, fig=fig)  # rate
    ax1 = plt.subplot2grid((8, 3), (3, 0), colspan=3, rowspan=5, fig=fig)  # data

    ax2 = ax1.twinx()
    ax1.set_xlabel("time (minutes)")
    ax1.set_ylabel("voltage (V vs. Li/Li+)", color=voltage_color)
    ax2.set_ylabel("current (mA)", color=current_color)

    annotations_1 = []  # step number (IR)
    annotations_2 = []  # step number
    annotations_4 = []  # rate

    for i, s in enumerate(all_steps):
        m = m_cycle_data & (data.Step_Index==s)
        c = data.loc[m, "Current"] * 1000
        v = data.loc[m, "Voltage"]
        t = data.loc[m, "Test_Time"]/60
        step_type, rate, current_max, dv, dc, d_discharge, d_charge = get_info(table, cycle, s)
        if len(t) > 1:
            fcolor = next(color)

            info_txt = f"{step_type}\nc-rate = {rate}\ni = |{1000*current_max:0.2f}| mA\n"
            info_txt += f"delta V = {dv:0.2f} %\ndelta i = {dc:0.2f} %\n"
            info_txt += f"delta C = {d_charge:0.2} %\ndelta DC = {d_discharge:0.2} %\n"

            for ax in [ax2, ax3, ax4]:
                ax.axvspan(t.iloc[0], t.iloc[-1], facecolor=fcolor, alpha=0.2)
            plot_step(ax1, t, v, voltage_color)
            plot_step(ax2, t, c, current_color)
            annotations_1.append([f"{s}", t.mean()])
            annotations_4.append([info_txt, t.mean()])
        else:
            info_txt = f"{s}({step_type})"
            annotations_2.append([info_txt, t.mean()])
    ax3.set_ylim(0,1)
    for s in annotations_1:
        ax3.annotate(f"{s[0]}", (s[1], 0.2), ha='center')

    for s in annotations_2:
        ax3.annotate(f"{s[0]}", (s[1], 0.6), ha='center')

    for s in annotations_4:
        ax4.annotate(f"{s[0]}", (s[1], 0.0), ha='center')

    for ax in [ax3, ax4]:
        ax.axes.get_yaxis().set_visible(False)
        ax.axes.get_xaxis().set_visible(False)
    
    #return (ax1, ax2, ax2, ax4)

In [ ]:
plot_raw_cycle(my_cell, 1)

In [ ]:
plot_raw_cycle(my_cell, 10)

In [ ]:
my_cell.raw_limits